In [ ]:
from lmptools import Dump, DumpCallback, DumpSnapshot
from lmptools.exceptions import SkipSnapshot
from typing import List
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
np.sort(np.random.choice(np.arange(9000000, 10002000, 2000), 20))

In [ ]:
class MyCallback(DumpCallback):
    def __init__(self, random_timesteps):
        self.timesteps = random_timesteps
        self.parsed_snapshots: List[DumpSnapshot] = []

    def on_snapshot_parse_timestamp(self, timestamp: int):
        if timestamp not in self.timesteps:
            raise SkipSnapshot(f"{timestamp} skipped")

    def on_snapshot_parse_end(self, snapshot: DumpSnapshot):
        print("snapshot parsed")
        self.parsed_snapshots.append(snapshot)


In [ ]:
d = Dump("dump.rigidChains.lammpstrj", callback=MyCallback(np.sort(np.random.choice(np.arange(9000000, 10002000, 2000), 20))))

In [ ]:
d.parse()